Research Track II - 2° assignment with jupyter integration

In [ ]:
%matplotlib notebook


import rospy
import actionlib
import actionlib.msg
import assignment_2_2022.msg
from std_srvs.srv import *
import sys
import select
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2022.msg import Position_velocity
from sensor_msgs.msg import LaserScan
import math
import ipywidgets as widgets
from IPython.display import display 
from ipywidgets import Layout, HBox
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, interactive, fixed, interact_manual
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np

goal_cancelled = 0
goal_reached = 0
canc_list, reach_list = [], []


Node Initialization

In [ ]:
# Inizialize the node
rospy.init_node('user_input')

Check Goal Status

In [ ]:
def CheckGoalStatus(msg):
    global goal_cancelled, goal_reached, canc_list, reach_list, goal

    # Get the status of the result from the msg 
    status = msg.status.status

    # Goal cancelled (status = 2)
    if status == 2:
        goal_cancelled += 1
        
        setting_button.disabled = False
        cancel_button.disabled = True
        
        x_position.disabled = False
        y_position.disabled = False
        
        canc_list.append((goal.target_pose.pose.position.x, goal.target_pose.pose.position.y))
       
        
        
    # Goal reached (status = 3)
    elif status == 3:
        goal_reached += 1
        
        setting_button.disabled = False
        cancel_button.disabled = True
        
        x_position.disabled = False
        y_position.disabled = False
        
        reach_list.append((goal.target_pose.pose.position.x, goal.target_pose.pose.position.y))
        
        
        

Graph depicting the trajectory of the robot and the status of goals (set, reached, deleted)

In [ ]:
global reach_list, canc_list, goal_cancelled, goal_reached

class Visualiser:
    def __init__(self):
        self.fig, (self.ax, self.ax2) = plt.subplots(2,1, figsize=(8,8))
        self.fig.subplots_adjust(hspace=0.5)
        self.path_line, = self.ax.plot([], [], 'b-', markersize=3, label='Robot Path')
        self.current_pos, = self.ax.plot([], [], 'ro', label='Current Position')
        self.target_pos, = self.ax.plot([], [], 'go', label='Target Position')
        self.old_target_pos, = self.ax.plot([], [], 'yo', label='Target Reached')
        self.cancelled_target_pos, = self.ax.plot([], [], 'mo', label='Target Cancelled')
        
        self.ax2.bar(['Reached', 'Deleted'], [goal_reached, goal_cancelled], color=['g', 'r'])

        self.x_data, self.y_data = [], []
        self.x_goal, self.y_goal = [], []
         
    def plot_init(self):
        self.ax.set_xlim(10, -10)
        self.ax.set_ylim(10, -10)
        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.grid(True)
        self.ax.legend() 
        
        self.ax2.grid(axis='y', linestyle='--', alpha=0.6)
        self.ax2.set_yticks(range(0, 11, 1))
        self.ax2.set_title('Goals statistics')
        self.ax2.set_xlabel('Goals')
        self.ax2.set_ylabel('Count')
        self.ax2.legend()
        
        return self.path_line, self.current_pos, self.target_pos, self.old_target_pos, self.cancelled_target_pos

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)     
        
        self.x_goal.append(goal.target_pose.pose.position.x)
        self.y_goal.append(goal.target_pose.pose.position.y)
        
    def update_plot(self, frame):
        self.path_line.set_data(self.x_data, self.y_data)
        self.current_pos.set_data(self.x_data[-1:], self.y_data[-1:])
        self.target_pos.set_data(self.x_goal[-1:], self.y_goal[-1:])
        self.old_target_pos.set_data([coord[0] for coord in reach_list], [coord[1] for coord in reach_list])
        self.cancelled_target_pos.set_data([coord[0] for coord in canc_list], [coord[1] for coord in canc_list])
        
        self.ax2.bar(['Reached', 'Deleted'], [goal_reached, goal_cancelled], color = ['g', 'r'])
        
        
        return self.path_line, self.current_pos, self.target_pos, self.old_target_pos, self.cancelled_target_pos

Function representing the current position and velocity of the robot

In [ ]:
def OdometryCallback(msg):

    global pub, current_x_widget, current_y_widget, vel_magnitude_widget#, position_x, position_y

    # Get the position from the msg
    pos = msg.pose.pose.position

    # Get the twist from the msg
    velocity = msg.twist.twist.linear

    # Create custom message
    position_velocity = Position_velocity()

    # Assign the parameters of the custom message
    position_velocity.x=pos.x
    position_velocity.y=pos.y
    position_velocity.v_x=velocity.x
    position_velocity.v_y=velocity.y
    
    # Update the current position widgets
    current_x_widget.value = round(pos.x, 2)
    current_y_widget.value = round(pos.y, 2)
    
    # Current magnitude velocity widget 
    vel_magnitude_widget.value = round(np.sqrt(position_velocity.v_x ** 2 + position_velocity.v_y ** 2), 2)
    
    # Publish the custom message
    pub.publish(position_velocity)

Functions for setting and deleting goals

In [ ]:
def setting_button_clicked(button):
    global x_position, y_position
    
    if cancel_button.disabled:
        
        # Enable cancel button when setting a new goal
        cancel_button.disabled = False
        setting_button.disabled = True
        
        # Diable FloatSlider widgets
        x_position.disabled = True
        y_position.disabled = True
        
        goal.target_pose.pose.position.x = x_position.value
        goal.target_pose.pose.position.y = y_position.value
        
        # Send the goal to the server
        client.send_goal(goal)
        
def cancel_button_clicked(button):
    global x_position, y_position
    
    if setting_button.disabled:
        
        # Enable setting button when cancelling the goal
        setting_button.disabled = False
        cancel_button.disabled = True
        
        # Enable FloatSlider widgets
        x_position.disabled = False
        y_position.disabled = False
    
        # Cancel goal
        client.cancel_goal()

Laser Scanner function

In [ ]:
def laserCallback(scan):
    valid_ranges = [x for x in scan.ranges if scan.range_min < x < scan.range_max]
    min_range = min(valid_ranges, default=100)
    angle = scan.angle_min + scan.angle_increment * scan.ranges.index(min_range)
    
    dmin.value = round(min_range, 2)
    ang.value = round(angle * (180.0 / math.pi), 2)

Widget creation

In [ ]:
# Create a text widget for displaying the targets and velocity magnitude
current_x_widget = widgets.FloatText(value=0.0, description='Current X position:', disabled=True, style={'description_width': 'initial', 'overflow': 'visible'})
current_y_widget = widgets.FloatText(value=0.0, description='Current Y position:', disabled=True, style={'description_width': 'initial', 'overflow': 'visible'})
vel_magnitude_widget = widgets.FloatText(value=0.0, description='Current Velocity:', disabled=True, style={'description_width': 'initial', 'overflow': 'visible'})

# FloatSlider widgets to choose the coordinates and Setting buttons
x_position = widgets.FloatSlider(value=0.0,min=-10.0,max=10.0,step=0.1,description='Set Target X:',orientation='horizontal')
y_position = widgets.FloatSlider(value=0.0,min=-10.0,max=10.0,step=0.1,description='Set Target Y:',orientation='horizontal')
setting_button = widgets.Button(value=False,description='SET GOAL',disabled=False,button_style='success',tooltip='Description')#'check') # (FontAwesome names without the `fa-` prefix))    
cancel_button = widgets.Button(value=False,description='CANC GOAL',disabled=True,button_style='danger',tooltip='Description')#'check') # (FontAwesome names without the `fa-` prefix))    

# Laser Widgets
dmin = widgets.FloatText(description = "Distance:", disabled = True)
ang = widgets.FloatText(description = "Angle:", disabled = True)

# Widget boxes to display
current_box = HBox([current_x_widget, current_y_widget, vel_magnitude_widget])
setting_box = HBox([x_position,y_position,setting_button, cancel_button])
laser_box = HBox([dmin, ang])

Publisher and Subscriber

In [ ]:
# PUBLISHER: send a message which contains two parameters (velocity and position)
pub = rospy.Publisher("/pos_vel", Position_velocity, queue_size = 1)

# Create the action client
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
    
# Wait for the server to be started
client.wait_for_server()

# Set goal position
goal = assignment_2_2022.msg.PlanningGoal()

global goal

# SUBSCRIBER: get from "Odom" two parameters (velocity and position)
sub_from_Odom=rospy.Subscriber("/odom",Odometry,OdometryCallback)

sub_from_scan=rospy.Subscriber("/scan", LaserScan, laserCallback)

vis = Visualiser()
rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, interval=100)
sub_from_goal = rospy.Subscriber("/reaching_goal/result", assignment_2_2022.msg.PlanningActionResult, CheckGoalStatus)
plt.show(block=True) 

Display Widgtes

In [ ]:
display(current_box)

setting_button.on_click(setting_button_clicked)
cancel_button.on_click(cancel_button_clicked)

display(setting_box)
display(laser_box)